# Dự đoán giá Ethereum trong tương lai 

In [5]:
import pandas as pd
import numpy as np 
import pandas as pd
import numpy as np
import math
import datetime as dt
import matplotlib.pyplot as plt
import seaborn as sns

from datetime import *

from sklearn.metrics import  mean_absolute_error, r2_score 
from sklearn.preprocessing import MinMaxScaler

import statsmodels.api as sm

# Models
from tensorflow.keras.layers import LSTM, Activation, Dense, Dropout, Conv1D, MaxPooling1D, Flatten, BatchNormalization
from tensorflow.keras.models import Sequential
from tensorflow.keras.losses import MSE
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.activations import relu

# proccessing 

Trong bài này sẽ sử dụng 4 bộ data 

data về giá eth : 

data về fed 

data về giá btc 

data về dxy 

In [6]:
df=pd.read_csv(r'C:\Users\HI\Desktop\predict-price-ethereum\data\eth_close.csv')
df1=pd.read_csv(r'C:\Users\HI\Desktop\predict-price-ethereum\data\btc_2025.csv')
df2=pd.read_csv(r'C:\Users\HI\Desktop\predict-price-ethereum\data\fedd.csv')
df3=pd.read_csv(r'C:\Users\HI\Desktop\predict-price-ethereum\data\dxy_data.csv',skiprows=2)

Data: giá ethereum

In [7]:
df.head()

,timestamp_utc,symbol,open,high,low,close,volume_eth,volume_usdt,trade_count
0,2021-12-31 17:00:00,ETHUSDT,3745.49,3754.26,3705.00,3722.03,12687.8551,4.729010e+07,30712
1,2021-12-31 18:00:00,ETHUSDT,3722.04,3725.01,3695.16,3704.43,10462.2432,3.880925e+07,23740
2,2021-12-31 19:00:00,ETHUSDT,3704.43,3705.93,3623.00,3626.27,20055.4006,7.338679e+07,50302
3,2021-12-31 20:00:00,ETHUSDT,3626.21,3663.87,3622.29,3645.04,12289.9700,4.485126e+07,30779
4,2021-12-31 21:00:00,ETHUSDT,3645.04,3701.85,3633.84,3688.41,13070.9293,4.792673e+07,32205


In [8]:
df.tail()

,timestamp_utc,symbol,open,high,low,close,volume_eth,volume_usdt,trade_count
31637,2025-08-10 22:00:00,ETHUSDT,4223.33,4250.63,4205.00,4250.63,12638.3173,5.341590e+07,97270
31638,2025-08-10 23:00:00,ETHUSDT,4250.63,4254.54,4228.14,4250.57,12440.1665,5.282397e+07,84149
31639,2025-08-11 00:00:00,ETHUSDT,4250.58,4284.99,4238.52,4262.94,25647.6173,1.094255e+08,214906
31640,2025-08-11 01:00:00,ETHUSDT,4262.94,4322.20,4261.00,4303.81,39750.4876,1.709397e+08,243998
31641,2025-08-11 02:00:00,ETHUSDT,4303.81,4343.00,4294.42,4309.05,44188.5938,1.907590e+08,300573


In [9]:
# ép date về đúng định dạng
df['timestamp_utc'] = pd.to_datetime(df['timestamp_utc']).dt.tz_localize('UTC')


In [10]:
# xem thông tin dữ liệu
print('Overview about data')
df.info()

Overview about data
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 31642 entries, 0 to 31641
Data columns (total 9 columns):
 #   Column         Non-Null Count  Dtype              
---  ------         --------------  -----              
 0   timestamp_utc  31642 non-null  datetime64[ns, UTC]
 1   symbol         31642 non-null  object             
 2   open           31642 non-null  float64            
 3   high           31642 non-null  float64            
 4   low            31642 non-null  float64            
 5   close          31642 non-null  float64            
 6   volume_eth     31642 non-null  float64            
 7   volume_usdt    31642 non-null  float64            
 8   trade_count    31642 non-null  int64              
dtypes: datetime64[ns, UTC](1), float64(6), int64(1), object(1)
memory usage: 2.2+ MB


In [11]:
df.isnull().sum()

timestamp_utc    0
symbol           0
open             0
high             0
low              0
close            0
volume_eth       0
volume_usdt      0
trade_count      0
dtype: int64

In [12]:
df = df[df['timestamp_utc'] < '2025-07-02']

In [13]:
df.tail()

,timestamp_utc,symbol,open,high,low,close,volume_eth,volume_usdt,trade_count
30674,2025-07-01 19:00:00+00:00,ETHUSDT,2410.14,2422.30,2402.34,2402.87,12821.8352,3.093746e+07,106687
30675,2025-07-01 20:00:00+00:00,ETHUSDT,2402.87,2418.57,2393.95,2415.08,19778.4342,4.757516e+07,102812
30676,2025-07-01 21:00:00+00:00,ETHUSDT,2415.08,2418.35,2404.22,2407.08,10647.6075,2.565205e+07,65246
30677,2025-07-01 22:00:00+00:00,ETHUSDT,2407.07,2413.29,2399.14,2400.66,10856.7699,2.612029e+07,65549
30678,2025-07-01 23:00:00+00:00,ETHUSDT,2400.66,2406.06,2386.52,2405.01,17455.7184,4.180945e+07,77217


data về btc

In [14]:
df1.head()

,timestamp_utc,close
0,2021-12-31 17:00:00,47016.57
1,2021-12-31 18:00:00,46686.41
2,2021-12-31 19:00:00,45728.28
3,2021-12-31 20:00:00,45879.24
4,2021-12-31 21:00:00,46333.86


In [15]:
df1.tail()

,timestamp_utc,close
30668,2025-07-01 13:00:00,106807.42
30669,2025-07-01 14:00:00,105925.00
30670,2025-07-01 15:00:00,105955.18
30671,2025-07-01 16:00:00,106082.56
30672,2025-07-01 17:00:00,106253.79


In [16]:
# kiểm tra định dạnh cột
df1.isnull().sum()

timestamp_utc    0
close            0
dtype: int64

In [17]:
# kiểm tra kiểu dữ liệu
df1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30673 entries, 0 to 30672
Data columns (total 2 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   timestamp_utc  30673 non-null  object 
 1   close          30673 non-null  float64
dtypes: float64(1), object(1)
memory usage: 479.4+ KB


In [18]:
# chuyển về kiểu dữ liệu datetime
df1['timestamp_utc'] = pd.to_datetime(df1['timestamp_utc']).dt.tz_localize('UTC')


In [19]:
df1.dtypes

timestamp_utc    datetime64[ns, UTC]
close                        float64
dtype: object

In [20]:
btc_df = df1.rename(columns={'timestamp_utc': 'timestamp_utc_btc'})

merged_df = pd.merge_asof(
    df,
    btc_df,
    left_on='timestamp_utc',
    right_on='timestamp_utc_btc',
    direction='backward',
    suffixes=('_ETH', '_BTC')
)




In [21]:
merged_df.head()

,timestamp_utc,symbol,open,high,low,close_ETH,volume_eth,volume_usdt,trade_count,timestamp_utc_btc,close_BTC
0,2021-12-31 17:00:00+00:00,ETHUSDT,3745.49,3754.26,3705.00,3722.03,12687.8551,4.729010e+07,30712,2021-12-31 17:00:00+00:00,47016.57
1,2021-12-31 18:00:00+00:00,ETHUSDT,3722.04,3725.01,3695.16,3704.43,10462.2432,3.880925e+07,23740,2021-12-31 18:00:00+00:00,46686.41
2,2021-12-31 19:00:00+00:00,ETHUSDT,3704.43,3705.93,3623.00,3626.27,20055.4006,7.338679e+07,50302,2021-12-31 19:00:00+00:00,45728.28
3,2021-12-31 20:00:00+00:00,ETHUSDT,3626.21,3663.87,3622.29,3645.04,12289.9700,4.485126e+07,30779,2021-12-31 20:00:00+00:00,45879.24
4,2021-12-31 21:00:00+00:00,ETHUSDT,3645.04,3701.85,3633.84,3688.41,13070.9293,4.792673e+07,32205,2021-12-31 21:00:00+00:00,46333.86


In [22]:
merged_df.drop(columns=['timestamp_utc_btc'], inplace=True)

In [23]:
merged_df.head()

,timestamp_utc,symbol,open,high,low,close_ETH,volume_eth,volume_usdt,trade_count,close_BTC
0,2021-12-31 17:00:00+00:00,ETHUSDT,3745.49,3754.26,3705.00,3722.03,12687.8551,4.729010e+07,30712,47016.57
1,2021-12-31 18:00:00+00:00,ETHUSDT,3722.04,3725.01,3695.16,3704.43,10462.2432,3.880925e+07,23740,46686.41
2,2021-12-31 19:00:00+00:00,ETHUSDT,3704.43,3705.93,3623.00,3626.27,20055.4006,7.338679e+07,50302,45728.28
3,2021-12-31 20:00:00+00:00,ETHUSDT,3626.21,3663.87,3622.29,3645.04,12289.9700,4.485126e+07,30779,45879.24
4,2021-12-31 21:00:00+00:00,ETHUSDT,3645.04,3701.85,3633.84,3688.41,13070.9293,4.792673e+07,32205,46333.86


data về fed 

In [24]:
merged_df['year_month'] = merged_df['timestamp_utc'].dt.strftime('%Y-%m')

In [25]:
df2.head()

,observation_date,FEDFUNDS
0,2021-12-01,0.08
1,2022-01-01,0.08
2,2022-02-01,0.08
3,2022-03-01,0.20
4,2022-04-01,0.33


Lãi suất đang theo từng tháng nên ta sẽ gán giá trị của tháng cho tất cả các ngày/giờ trong tháng đó.

In [26]:
df2.isnull().sum()

observation_date    0
FEDFUNDS            0
dtype: int64

In [27]:
df2.dtypes

observation_date     object
FEDFUNDS            float64
dtype: object

In [28]:
# đưa về kiểu dữ liệu chuẩn utc
df2['observation_date'] = pd.to_datetime(df2['observation_date']).dt.tz_localize('UTC')

In [29]:
df2['year_month'] = df2['observation_date'].dt.strftime('%Y-%m')

In [30]:
df2.columns

Index(['observation_date', 'FEDFUNDS', 'year_month'], dtype='object')

In [31]:
merged_df = pd.merge(merged_df, df2[['year_month', 'FEDFUNDS']], on='year_month', how='left')

In [32]:
merged_df.head()

,timestamp_utc,symbol,open,high,low,close_ETH,volume_eth,volume_usdt,trade_count,close_BTC,year_month,FEDFUNDS
0,2021-12-31 17:00:00+00:00,ETHUSDT,3745.49,3754.26,3705.00,3722.03,12687.8551,4.729010e+07,30712,47016.57,2021-12,0.08
1,2021-12-31 18:00:00+00:00,ETHUSDT,3722.04,3725.01,3695.16,3704.43,10462.2432,3.880925e+07,23740,46686.41,2021-12,0.08
2,2021-12-31 19:00:00+00:00,ETHUSDT,3704.43,3705.93,3623.00,3626.27,20055.4006,7.338679e+07,50302,45728.28,2021-12,0.08
3,2021-12-31 20:00:00+00:00,ETHUSDT,3626.21,3663.87,3622.29,3645.04,12289.9700,4.485126e+07,30779,45879.24,2021-12,0.08
4,2021-12-31 21:00:00+00:00,ETHUSDT,3645.04,3701.85,3633.84,3688.41,13070.9293,4.792673e+07,32205,46333.86,2021-12,0.08


In [33]:
merged_df.tail()

,timestamp_utc,symbol,open,high,low,close_ETH,volume_eth,volume_usdt,trade_count,close_BTC,year_month,FEDFUNDS
30674,2025-07-01 19:00:00+00:00,ETHUSDT,2410.14,2422.30,2402.34,2402.87,12821.8352,3.093746e+07,106687,106253.79,2025-07,4.33
30675,2025-07-01 20:00:00+00:00,ETHUSDT,2402.87,2418.57,2393.95,2415.08,19778.4342,4.757516e+07,102812,106253.79,2025-07,4.33
30676,2025-07-01 21:00:00+00:00,ETHUSDT,2415.08,2418.35,2404.22,2407.08,10647.6075,2.565205e+07,65246,106253.79,2025-07,4.33
30677,2025-07-01 22:00:00+00:00,ETHUSDT,2407.07,2413.29,2399.14,2400.66,10856.7699,2.612029e+07,65549,106253.79,2025-07,4.33
30678,2025-07-01 23:00:00+00:00,ETHUSDT,2400.66,2406.06,2386.52,2405.01,17455.7184,4.180945e+07,77217,106253.79,2025-07,4.33


với data về dxy

In [34]:
df3.head()

,Date,Unnamed: 1
0,2021-12-31,95.669998
1,2022-01-03,96.239998
2,2022-01-04,96.290001
3,2022-01-05,96.190002
4,2022-01-06,96.250000


thị trường tài chính dxy không hoạt động như crypto nó chỉ hoạt động 24/7 và sẽ đóng cửa vào ngày cuối tuần hoặc lễ nên nó sẽ có trường hợp bị nan , ta sẽ xử lý bằng cách điền giá trị ngày gần nhất cho nó , với trường hợp mà ngày đầu tiên bị nan thì ta sẽ lấy ngày đầu tiên tháng mới điền ngược lại 

In [35]:
df3.isnull().sum()

Date          0
Unnamed: 1    0
dtype: int64

In [36]:
# kiểm tra kiểu dữ liêu
df3.dtypes

Date           object
Unnamed: 1    float64
dtype: object

In [37]:
# chuyển về dạng datetime utc
df3['Date'] = pd.to_datetime(df3['Date']).dt.tz_localize('UTC')

trích xuất thêm cột date từ timestamp_utc và tạo thêm cột date trong df3 để làm khóa chính 

In [38]:
merged_df['Date'] = merged_df['timestamp_utc'].dt.date

In [39]:
merged_df.head()

,timestamp_utc,symbol,open,high,low,close_ETH,volume_eth,volume_usdt,trade_count,close_BTC,year_month,FEDFUNDS,Date
0,2021-12-31 17:00:00+00:00,ETHUSDT,3745.49,3754.26,3705.00,3722.03,12687.8551,4.729010e+07,30712,47016.57,2021-12,0.08,2021-12-31
1,2021-12-31 18:00:00+00:00,ETHUSDT,3722.04,3725.01,3695.16,3704.43,10462.2432,3.880925e+07,23740,46686.41,2021-12,0.08,2021-12-31
2,2021-12-31 19:00:00+00:00,ETHUSDT,3704.43,3705.93,3623.00,3626.27,20055.4006,7.338679e+07,50302,45728.28,2021-12,0.08,2021-12-31
3,2021-12-31 20:00:00+00:00,ETHUSDT,3626.21,3663.87,3622.29,3645.04,12289.9700,4.485126e+07,30779,45879.24,2021-12,0.08,2021-12-31
4,2021-12-31 21:00:00+00:00,ETHUSDT,3645.04,3701.85,3633.84,3688.41,13070.9293,4.792673e+07,32205,46333.86,2021-12,0.08,2021-12-31


In [40]:
merged_df['Date'] = pd.to_datetime(merged_df['Date']).dt.tz_localize('UTC')

In [41]:
merged_df.head()

,timestamp_utc,symbol,open,high,low,close_ETH,volume_eth,volume_usdt,trade_count,close_BTC,year_month,FEDFUNDS,Date
0,2021-12-31 17:00:00+00:00,ETHUSDT,3745.49,3754.26,3705.00,3722.03,12687.8551,4.729010e+07,30712,47016.57,2021-12,0.08,2021-12-31 00:00:00+00:00
1,2021-12-31 18:00:00+00:00,ETHUSDT,3722.04,3725.01,3695.16,3704.43,10462.2432,3.880925e+07,23740,46686.41,2021-12,0.08,2021-12-31 00:00:00+00:00
2,2021-12-31 19:00:00+00:00,ETHUSDT,3704.43,3705.93,3623.00,3626.27,20055.4006,7.338679e+07,50302,45728.28,2021-12,0.08,2021-12-31 00:00:00+00:00
3,2021-12-31 20:00:00+00:00,ETHUSDT,3626.21,3663.87,3622.29,3645.04,12289.9700,4.485126e+07,30779,45879.24,2021-12,0.08,2021-12-31 00:00:00+00:00
4,2021-12-31 21:00:00+00:00,ETHUSDT,3645.04,3701.85,3633.84,3688.41,13070.9293,4.792673e+07,32205,46333.86,2021-12,0.08,2021-12-31 00:00:00+00:00


In [42]:
df3.head()

,Date,Unnamed: 1
0,2021-12-31 00:00:00+00:00,95.669998
1,2022-01-03 00:00:00+00:00,96.239998
2,2022-01-04 00:00:00+00:00,96.290001
3,2022-01-05 00:00:00+00:00,96.190002
4,2022-01-06 00:00:00+00:00,96.250000


In [43]:

df = merged_df.sort_values('timestamp_utc')
df3 = df3.sort_values('Date')
df['timestamp_utc'] = df['timestamp_utc'].dt.tz_convert(None)
df3['Date'] = df3['Date'].dt.tz_convert(None)
merged_df = pd.merge_asof(
    df,
    df3,
    left_on='timestamp_utc',
    right_on='Date',
    direction='backward'  
)



In [44]:
merged_df.head()

,timestamp_utc,symbol,open,high,low,close_ETH,volume_eth,volume_usdt,trade_count,close_BTC,year_month,FEDFUNDS,Date_x,Date_y,Unnamed: 1
0,2021-12-31 17:00:00,ETHUSDT,3745.49,3754.26,3705.00,3722.03,12687.8551,4.729010e+07,30712,47016.57,2021-12,0.08,2021-12-31 00:00:00+00:00,2021-12-31,95.669998
1,2021-12-31 18:00:00,ETHUSDT,3722.04,3725.01,3695.16,3704.43,10462.2432,3.880925e+07,23740,46686.41,2021-12,0.08,2021-12-31 00:00:00+00:00,2021-12-31,95.669998
2,2021-12-31 19:00:00,ETHUSDT,3704.43,3705.93,3623.00,3626.27,20055.4006,7.338679e+07,50302,45728.28,2021-12,0.08,2021-12-31 00:00:00+00:00,2021-12-31,95.669998
3,2021-12-31 20:00:00,ETHUSDT,3626.21,3663.87,3622.29,3645.04,12289.9700,4.485126e+07,30779,45879.24,2021-12,0.08,2021-12-31 00:00:00+00:00,2021-12-31,95.669998
4,2021-12-31 21:00:00,ETHUSDT,3645.04,3701.85,3633.84,3688.41,13070.9293,4.792673e+07,32205,46333.86,2021-12,0.08,2021-12-31 00:00:00+00:00,2021-12-31,95.669998


In [45]:
merged_df.tail(5)

,timestamp_utc,symbol,open,high,low,close_ETH,volume_eth,volume_usdt,trade_count,close_BTC,year_month,FEDFUNDS,Date_x,Date_y,Unnamed: 1
30674,2025-07-01 19:00:00,ETHUSDT,2410.14,2422.30,2402.34,2402.87,12821.8352,3.093746e+07,106687,106253.79,2025-07,4.33,2025-07-01 00:00:00+00:00,2025-07-01,96.82
30675,2025-07-01 20:00:00,ETHUSDT,2402.87,2418.57,2393.95,2415.08,19778.4342,4.757516e+07,102812,106253.79,2025-07,4.33,2025-07-01 00:00:00+00:00,2025-07-01,96.82
30676,2025-07-01 21:00:00,ETHUSDT,2415.08,2418.35,2404.22,2407.08,10647.6075,2.565205e+07,65246,106253.79,2025-07,4.33,2025-07-01 00:00:00+00:00,2025-07-01,96.82
30677,2025-07-01 22:00:00,ETHUSDT,2407.07,2413.29,2399.14,2400.66,10856.7699,2.612029e+07,65549,106253.79,2025-07,4.33,2025-07-01 00:00:00+00:00,2025-07-01,96.82
30678,2025-07-01 23:00:00,ETHUSDT,2400.66,2406.06,2386.52,2405.01,17455.7184,4.180945e+07,77217,106253.79,2025-07,4.33,2025-07-01 00:00:00+00:00,2025-07-01,96.82


ta sẽ bỏ cột symbol, date_x, date_y , year_month, trade_count vì nó không có giá trị cho phân tích 

cột volume_eth và colume_usdt thật ra là cùng đo lường khối lượng giao dịch nhưng chỉ khác đơn vị 

In [46]:
columns_ = ['symbol','Date_x','Date_y','year_month','trade_count']
df_cleaned = merged_df.drop(columns=columns_)

In [47]:
df_cleaned.head()

,timestamp_utc,open,high,low,close_ETH,volume_eth,volume_usdt,close_BTC,FEDFUNDS,Unnamed: 1
0,2021-12-31 17:00:00,3745.49,3754.26,3705.00,3722.03,12687.8551,4.729010e+07,47016.57,0.08,95.669998
1,2021-12-31 18:00:00,3722.04,3725.01,3695.16,3704.43,10462.2432,3.880925e+07,46686.41,0.08,95.669998
2,2021-12-31 19:00:00,3704.43,3705.93,3623.00,3626.27,20055.4006,7.338679e+07,45728.28,0.08,95.669998
3,2021-12-31 20:00:00,3626.21,3663.87,3622.29,3645.04,12289.9700,4.485126e+07,45879.24,0.08,95.669998
4,2021-12-31 21:00:00,3645.04,3701.85,3633.84,3688.41,13070.9293,4.792673e+07,46333.86,0.08,95.669998


In [48]:
df_cleaned.rename(columns={'Unnamed: 1': 'dxy'}, inplace=True)
df_cleaned.columns = df_cleaned.columns.str.lower()

In [49]:
df_cleaned.head()

,timestamp_utc,open,high,low,close_eth,volume_eth,volume_usdt,close_btc,fedfunds,dxy
0,2021-12-31 17:00:00,3745.49,3754.26,3705.00,3722.03,12687.8551,4.729010e+07,47016.57,0.08,95.669998
1,2021-12-31 18:00:00,3722.04,3725.01,3695.16,3704.43,10462.2432,3.880925e+07,46686.41,0.08,95.669998
2,2021-12-31 19:00:00,3704.43,3705.93,3623.00,3626.27,20055.4006,7.338679e+07,45728.28,0.08,95.669998
3,2021-12-31 20:00:00,3626.21,3663.87,3622.29,3645.04,12289.9700,4.485126e+07,45879.24,0.08,95.669998
4,2021-12-31 21:00:00,3645.04,3701.85,3633.84,3688.41,13070.9293,4.792673e+07,46333.86,0.08,95.669998


In [50]:
df_cleaned.isnull().sum()

timestamp_utc    0
open             0
high             0
low              0
close_eth        0
volume_eth       0
volume_usdt      0
close_btc        0
fedfunds         0
dxy              0
dtype: int64

Xem thống kê cơ bản 

In [51]:
df_numeric = df_cleaned.select_dtypes(exclude=['datetime64[ns]'])
df_numeric.describe()

,open,high,low,close_eth,volume_eth,volume_usdt,close_btc,fedfunds,dxy
count,30679.000000,30679.000000,30679.000000,30679.000000,30679.000000,3.067900e+04,30679.000000,30679.000000,30679.000000
mean,2299.126406,2310.029127,2287.603579,2299.082850,21310.621565,4.673171e+07,48838.725416,4.007779,103.768024
std,752.414506,756.033233,748.295933,752.369588,24118.047757,5.131673e+07,27375.089562,1.682695,3.436043
min,904.260000,928.360000,881.560000,904.250000,0.000000,0.000000e+00,15649.520000,0.080000,94.790001
25%,1667.580000,1674.830000,1661.970000,1667.570000,8255.014850,1.801160e+07,26477.375000,3.780000,101.919998
50%,2192.210000,2207.090000,2176.900000,2192.200000,13926.842600,3.150289e+07,40742.920000,4.640000,103.919998
75%,2919.600000,2932.015000,2905.060000,2919.580000,25257.921750,5.657883e+07,66890.285000,5.330000,105.739998
max,4086.660000,4107.800000,4062.600000,4086.660000,508838.404900,1.245779e+09,111780.840000,5.330000,114.110001


vì mục đích là dự đoán xu hướng tăng giảm thì ta sẽ cần một số chỉ số kĩ thuật đo động lượng và đo xu hướng 

MACD(moving Average Convergence divergence) dựa trên sự chênh lệch giữa các đường trung bình động hàm mũ, cho tín hiệu xu hướng và điểm đảo chiều 

EMA (exponential moving average): trung bình động hàm mũ, làm mượt dữ liệu giá cho thấy xu hướng hiện tại với trọng số cao cho dữ liệu mới 

RSi (Relative Strength Index): đo sức mạnh xu hướng hiện tại, giá trị từ 0 đến 100 thường dùng mức 30, 70 để xác định quá mua / quá bán 

In [52]:
df_cleaned['ema_12'] = df_cleaned['close_eth'].ewm(span=12, adjust=False).mean()
df_cleaned['ema_26'] = df_cleaned['close_eth'].ewm(span=26, adjust=False).mean()

# Tính MACD và đường tín hiệu MACD
df_cleaned['macd'] = df_cleaned['ema_12'] - df_cleaned['ema_26']
df_cleaned['macd_signal'] = df_cleaned['macd'].ewm(span=9, adjust=False).mean()

# Tính RSI 14
delta = df_cleaned['close_eth'].diff()
gain = delta.clip(lower=0)
loss = -delta.clip(upper=0)
avg_gain = gain.rolling(window=14).mean()
avg_loss = loss.rolling(window=14).mean()
rs = avg_gain / avg_loss
df_cleaned['rsi_14'] = 100 - (100 / (1 + rs))

In [53]:
#
df_cleaned = df_cleaned.dropna()


In [54]:
df_cleaned.head()

,timestamp_utc,open,high,low,close_eth,volume_eth,volume_usdt,close_btc,fedfunds,dxy,ema_12,ema_26,macd,macd_signal,rsi_14
14,2022-01-01 07:00:00,3711.93,3724.93,3701.00,3715.31,6541.6499,2.428604e+07,47194.73,0.08,95.669998,3712.289733,3711.268607,1.021126,-2.217128,48.867772
15,2022-01-01 08:00:00,3715.32,3733.84,3708.60,3721.40,4013.6403,1.493429e+07,47124.82,0.08,95.669998,3713.691312,3712.019080,1.672232,-1.439256,52.974584
16,2022-01-01 09:00:00,3721.14,3730.00,3709.38,3718.67,4515.6636,1.680313e+07,47143.98,0.08,95.669998,3714.457264,3712.511741,1.945523,-0.762300,72.018873
17,2022-01-01 10:00:00,3718.67,3731.88,3673.46,3691.39,10412.4737,3.851047e+07,46871.08,0.08,95.669998,3710.908454,3710.947168,-0.038713,-0.617583,60.614666
18,2022-01-01 11:00:00,3691.39,3712.19,3691.02,3693.37,4786.2173,1.771969e+07,46758.87,0.08,95.669998,3708.210231,3709.645155,-1.434925,-0.781051,51.401605


In [55]:
df_cleaned.shape

(30665, 15)

# trực quan hóa dữ liệu

## phân tích giá eth qua thời gian 


In [56]:
df=df_cleaned.copy()

In [57]:
import pandas as pd
import plotly.graph_objects as go
fig = go.Figure()

# Giá đóng ETH
fig.add_trace(go.Scatter(
    x=df['timestamp_utc'],
    y=df['close_eth'],
    mode='lines',
    name='Close ETH',
    line=dict(color='skyblue', width=1.5)
))

# Layout
fig.update_layout(
    title='Giá đóng ETH theo thời gian',
    xaxis_title='Thời gian ',
    yaxis_title='Giá ETH (USD)',
    hovermode='x unified',
    width = 800,
    height= 500)
fig.show()


với biểu đồ này ta có thể thấy được là trong giai đoạn từ tháng 1/2022 đến cuối tháng 6/2025 giá ETH giao dộng khá mạnh 

Đầu năm 2022 giá đang ở mức khoảng 3800 usd nhưng cho đến thời điểm nửa cuối năm 2022( tháng 7) giá eth trải qua một đợt giảm mạnh, mất hơn 50% giá trị từ vùng trên $3500 xuống gần $1500

Cuối năm 2022 giá bắt đầu tăng nhẹ khoảng 1270 chủ yếu xoay quanh mốc $1500 - $2000 tới cuối năm 2023 vẫn không biến động nhiều. Đây là giai đoạn tạo đáy và tích lũy, cho thấy áp lực bán đã yếu đi và thị trường đang tìm lại sự cân bằng.

Cuối năm 2023 - cuối 2024 : thị trường bước vào xu hướng tăng mới , đỉnh điểm là giá cao nhất tại tháng 16/12/2024 là 4086 usd sau giai đoạn này đến 6/4/2025 giá lại giảm mạnh xuống chỉ còn 1418 usd và tính cho tới thời điểm hiện tại theo data giá của eth là 2493 usd 

- Giai đoạn biến động mạnh nhất rõ ràng là từ cuối năm 2023 đến đầu năm 2025.

rút ra : 

giá có thể lặp lại chu trình Giảm -> Đi ngang -> Tăng -> Giảm

thời điểm mua tốt nhất là lúc thị trường đi ngang 





## phân tích giá eth theo khối lượng giao dịch 

In [58]:
df_month = df.resample('M', on='timestamp_utc').agg({
    'close_eth': 'last',    
    'volume_eth': 'sum'     
}).reset_index()

df_month.head()

C:\Users\HI\AppData\Local\Temp\ipykernel_20752\1529404397.py:1: FutureWarning:

'M' is deprecated and will be removed in a future version, please use 'ME' instead.



,timestamp_utc,close_eth,volume_eth
0,2022-01-31,2686.94,1.391925e+07
1,2022-02-28,2920.95,1.413168e+07
2,2022-03-31,3281.51,1.426624e+07
3,2022-04-30,2726.66,1.294994e+07
4,2022-05-31,1941.90,2.477743e+07


In [59]:
import plotly.graph_objects as go

# Tạo figure
fig = go.Figure()

# Giá đóng ETH (trục chính bên trái)
fig.add_trace(go.Scatter(
    x=df_month['timestamp_utc'],
    y=df_month['close_eth'],
    mode='lines',
    name='Close ETH',
    line=dict(color='skyblue', width=1.5),
    yaxis='y1'
))

# Volume ETH (trục phụ bên phải)
fig.add_trace(go.Scatter(
    x=df_month['timestamp_utc'],
    y=df_month['volume_eth'],
    mode='lines',
    name='Volume ETH',
    line=dict(color='orange', width=1.5, dash='dot'),
    yaxis='y2'
))

# Layout
fig.update_layout(
    title='Giá đóng và Khối lượng giao dịch ETH theo thời gian',
    xaxis_title='Thời gian',
    yaxis=dict(
        title='Giá ETH (USD)',
        side='left',
        showgrid=False
    ),
    yaxis2=dict(
        title='Khối lượng (ETH)',
        side='right',
        overlaying='y',
        showgrid=False
    ),
    hovermode='x unified',
    width = 800 ,
    height = 500

)

fig.show()


Nhìn vào biểu đồ này ta thấy được từ đầu năm 2022 giá eth có xu hướng giảm mạnh đến tháng 7/2022 giá giảm tới mức 1071 usd , và củng ở thời điểm này khối lượng giao dịch củng cao đỉnh điểm đạt 40 triệu giao dịch eth , ở đây xảy ra 2 trường hợp 

- một số nhà đầu tư sẽ bán ra để cắt lỗ khi đã mua với giá cao trước đó 

- một số người sẽ có tầm nhìn dài hạn xem đây là cơ hội để mua vào tài sản tốt với giá rẻ 

giai đoạn năm 2023 : giá có xu hướng đi ngang và tăng nhẹ trong khi khối lượng duy trì ở mức thấp và giảm dần -> kl thấp thì cho thấy thị trường đang"chán nản" đây có thể là thời điểm lý tưởng để mua tích lũy 

giai đoạn năm 2024 : giá bắt đầu tăng và khối lượng gd tăng dần , ta thấy được xu hướng tăng , tạo niềm tin và sức đẩy giá tiếp tục đi lên 

cuối tháng 4 năm 2025 giá lại giảm 1793 usd và khối lượng giao dịch vẫn tiếp tục tăng nhẹ 19 triệu giao dịch sau khoảng thời gian này giá eth tăng lên 2528 thì khối lượng giao dịch củng tăng thêm khoảng 2 triệu giao dịch , và sau đó tới 1/7/2025 giá giảm nhẹ thì khối lượng giao dịch giảm xuống khoảng 351 nghìn eth 

--> Đây là một tín hiệu cực kỳ mạnh mẽ cho thấy phe bán đã hoàn toàn kiệt sức. Không còn ai muốn bán ở mức giá này nữa. Đây thường là dấu hiệu báo trước một sự đảo chiều tăng giá sắp xảy ra.



## phân tích giá eth qua 1 số chỉ báo kĩ thuật 

In [60]:
def compute_rsi(series, window=14):
    delta = series.diff()
    gain = delta.clip(lower=0)
    loss = -delta.clip(upper=0)
    avg_gain = gain.rolling(window=window, min_periods=window).mean()
    avg_loss = loss.rolling(window=window, min_periods=window).mean()
    rs = avg_gain / avg_loss
    rsi = 100 - (100 / (1 + rs))
    return rsi
df_rsi_thang= df.resample('M', on='timestamp_utc')['close_eth'].last().reset_index()
df_rsi_thang['rsi_14'] = compute_rsi(df_rsi_thang['close_eth'], window=14)



C:\Users\HI\AppData\Local\Temp\ipykernel_20752\2903050629.py:10: FutureWarning:

'M' is deprecated and will be removed in a future version, please use 'ME' instead.



In [61]:
df_rsi_thang.head()

,timestamp_utc,close_eth,rsi_14
0,2022-01-31,2686.94,NaN
1,2022-02-28,2920.95,NaN
2,2022-03-31,3281.51,NaN
3,2022-04-30,2726.66,NaN
4,2022-05-31,1941.90,NaN


In [62]:
df_rsi_thang = df_rsi_thang.dropna(subset=['rsi_14'])

In [63]:
df_rsi_thang.isnull().sum()

timestamp_utc    0
close_eth        0
rsi_14           0
dtype: int64

In [64]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# Tạo subplot: 2 hàng, 1 cột, chia trục x chung
fig = make_subplots(
    rows=2, cols=1,
    shared_xaxes=True,
    vertical_spacing=0.05,
    subplot_titles=("Giá đóng ETH", "RSI 14")
)

# --- Hàng 1: Giá đóng ETH ---
fig.add_trace(go.Scatter(
    x=df_rsi_thang['timestamp_utc'],
    y=df_rsi_thang['close_eth'],
    mode='lines',
    name='Close ETH',
    line=dict(color='skyblue', width=1.5)
), row=1, col=1)

# --- Hàng 2: RSI ---
fig.add_trace(go.Scatter(
    x=df_rsi_thang['timestamp_utc'],
    y=df_rsi_thang['rsi_14'],
    mode='lines',
    name='RSI 14',
    line=dict(color='orange', width=1.5)
), row=2, col=1)

# Đường ngang ở mức 70 (quá mua)
fig.add_hline(y=70, line_dash="dot", line_color="red", row=2, col=1)
# Đường ngang ở mức 30 (quá bán)
fig.add_hline(y=30, line_dash="dot", line_color="green", row=2, col=1)

# Layout chung
fig.update_layout(
    height=500,
    title="Giá ETH và RSI 14",
   # template="plotly_dark",
    hovermode="x unified",
    yaxis_title="Giá ETH (USD)",
    yaxis2_title="RSI",
    width = 600
)

fig.show()


chỉ số rsi 14 là chỉ số dùng để đo sự thay đổi của giá qua 14 phiên gần nhất 

quá mua : là trạng thái khi giá của một tài sản đã tăng quá nhanh và quá nhiều trong một khoảng thời gian ngắn. Điều này cho thấy tài sản đó đang bị mua vào một cách thái quá và có thể sắp bước vào giai đoạn điều chỉnh giảm giá

quá bán : Quá bán là trạng thái khi giá của một tài sản đã giảm quá nhanh và quá nhiều trong một khoảng thời gian ngắn. Điều này cho thấy tài sản đó đang bị bán ra một cách thái quá và có thể sắp phục hồi tăng giá.

Khi RSI vượt 70 → Thường trùng với các giai đoạn giá ETH đạt đỉnh hoặc gần đỉnh ngắn hạn → sau đó giá có xu hướng giảm hoặc điều chỉnh.

Khi RSI xuống dưới 30 → Thường trùng với các đáy giá ETH → sau đó giá thường bật tăng

1 vài thời điểm nổi bật : 

cuối tháng 2/2024 chỉ số rsi > 70 tại thời điểm đó thì giá đang ở mức 3340 và có xu hướng tăng , lực mua đáng chiếm ưu thế khiến tài sản bị định giá cao trong ngắn hạn , tín hiện này cảnh báo là xu hướng tăng có thể không bền vững , sắp có đợt điều chỉnh giá 



In [65]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# Resample lấy giá đóng cuối tháng
df_month = df.resample('M', on='timestamp_utc').agg({'close_eth': 'last'}).reset_index()

# Tính EMA 10 tháng, EMA 26 tháng
df_month['ema_10'] = df_month['close_eth'].ewm(span=10, adjust=False).mean()
df_month['ema_26'] = df_month['close_eth'].ewm(span=26, adjust=False).mean()

# Tính MACD và MACD signal
df_month['macd'] = df_month['ema_10'] - df_month['ema_26']
df_month['macd_signal'] = df_month['macd'].ewm(span=9, adjust=False).mean()

# --- 2. Xác định điểm MACD cắt lên/cắt xuống ---
df_month['macd_diff'] = df_month['macd'] - df_month['macd_signal']
df_month['macd_diff_prev'] = df_month['macd_diff'].shift(1)

df_month['cross_up'] = (df_month['macd_diff'] > 0) & (df_month['macd_diff_prev'] <= 0)
df_month['cross_down'] = (df_month['macd_diff'] < 0) & (df_month['macd_diff_prev'] >= 0)

# --- 3. Tạo subplot ---
fig = make_subplots(
    rows=2, cols=1,
    shared_xaxes=True,
    vertical_spacing=0.05,
    subplot_titles=("Giá ETH", "MACD & MACD Signal")
)

# --- Hàng 1: Giá ETH ---
fig.add_trace(go.Scatter(
    x=df_month['timestamp_utc'],
    y=df_month['close_eth'],
    mode='lines',
    name='Close ETH',
    line=dict(color='skyblue', width=1.5)
), row=1, col=1)

# --- Hàng 2: MACD & Signal ---
fig.add_trace(go.Scatter(
    x=df_month['timestamp_utc'],
    y=df_month['macd'],
    mode='lines',
    name='MACD',
    line=dict(color='orange', width=1.5)
), row=2, col=1)

fig.add_trace(go.Scatter(
    x=df_month['timestamp_utc'],
    y=df_month['macd_signal'],
    mode='lines',
    name='Signal',
    line=dict(color='green', width=1.5, dash='dot')
), row=2, col=1)

# --- Hàng 2: Đánh dấu điểm cắt lên ---
fig.add_trace(go.Scatter(
    x=df_month.loc[df_month['cross_up'], 'timestamp_utc'],
    y=df_month.loc[df_month['cross_up'], 'macd'],
    mode='markers',
    name='MACD Cross Up',
    marker=dict(symbol='triangle-up', color='red', size=10)
), row=2, col=1)

# --- Hàng 2: Đánh dấu điểm cắt xuống ---
fig.add_trace(go.Scatter(
    x=df_month.loc[df_month['cross_down'], 'timestamp_utc'],
    y=df_month.loc[df_month['cross_down'], 'macd'],
    mode='markers',
    name='MACD Cross Down',
    marker=dict(symbol='triangle-down', color='blue', size=10)
), row=2, col=1)

# --- Layout ---
fig.update_layout(
    height=500,
    width=800,
    title="MACD Crossovers & Giá ETH (Resample Tháng)",
    hovermode="x unified"
)

fig.show()


C:\Users\HI\AppData\Local\Temp\ipykernel_20752\2656373761.py:5: FutureWarning:

'M' is deprecated and will be removed in a future version, please use 'ME' instead.



In [66]:
import plotly.express as px
df_month = df.resample('M', on='timestamp_utc').agg({
    'close_eth': 'last',
    'close_btc': 'last'
}).reset_index()

# --- 2. Tính hệ số tương quan ---
correlation = df_month['close_eth'].corr(df_month['close_btc'])
print(f"Hệ số tương quan ETH & BTC: {correlation:.2f}")

# --- 3. Vẽ biểu đồ ---
fig = go.Figure()

# Giá ETH (trục trái)
fig.add_trace(go.Scatter(
    x=df_month['timestamp_utc'],
    y=df_month['close_eth'],
    mode='lines',
    name='ETH',
    line=dict(color='skyblue', width=1.5),
    yaxis='y1'
))

# Giá BTC (trục phải)
fig.add_trace(go.Scatter(
    x=df_month['timestamp_utc'],
    y=df_month['close_btc'],
    mode='lines',
    name='BTC',
    line=dict(color='orange', width=1.5),
    yaxis='y2'
))

# Layout
fig.update_layout(
    title=f'Biểu đồ so sánh giá ETH và BTC (Hệ số tương quan = {correlation:.2f})',
    xaxis_title='Thời gian',
    yaxis=dict(
        title='Giá ETH (USD)',
        side='left',
        showgrid=False
    ),
    yaxis2=dict(
        title='Giá BTC (USD)',
        side='right',
        overlaying='y',
        showgrid=False
    ),
    hovermode='x unified',
    #template='plotly_dark',
    width=800,
    height=500
)

fig.show()

Hệ số tương quan ETH & BTC: 0.63


C:\Users\HI\AppData\Local\Temp\ipykernel_20752\970796499.py:2: FutureWarning:

'M' is deprecated and will be removed in a future version, please use 'ME' instead.



khi BTC tăng, ETH cũng có xu hướng tăng, và khi BTC giảm, ETH cũng giảm theo, ta có thể thấy được là bct và eth có mối quan hệ tương quan khá cao 

khoảng giữa 2022: BTC giảm manh, ETH củng lao dốc gần như cùng lúc với tỷ lệ giảm sâu hơn 

Đầu 2023 và đầu 2024: BTC tăng → ETH cũng tăng theo, thường với biên độ tăng tương đối lớn


--> nhà đầu tư nên theo dõi btc như 1 chỉ báo sớm, nếu btc phá xu hướng khả năng cao eth sẽ phản ứng tương tự trong vài ngày đến 

In [67]:

import plotly.express as px# --- 1. Resample theo tháng ---
df_month = df.resample('M', on='timestamp_utc').agg({
    'close_eth': 'last',
    'fedfunds': 'last'
}).reset_index()
# tính hệ số tương quan 
corr_fed = df_month['close_eth'].corr(df_month['fedfunds'])
print(f"Hệ số tương quan ETH & Fed Funds: {corr_fed:.2f}")


# --- 4. Scatter plot ETH vs Fed Funds ---
fig2 = px.scatter(
    df_month, x='fedfunds', y='close_eth',
    trendline='ols',
    title=f'Mối quan hệ ETH và Lãi suất FED (Corr={corr_fed:.2f})',
    labels={'fedfunds': 'Lãi suất FED (%)', 'close_eth': 'Giá ETH (USD)'},
    #template='plotly_dark'
)
fig2.show()


C:\Users\HI\AppData\Local\Temp\ipykernel_20752\189473428.py:2: FutureWarning:

'M' is deprecated and will be removed in a future version, please use 'ME' instead.



Hệ số tương quan ETH & Fed Funds: 0.11


- biểu đồ fed 

Đường hồi quy có độ dốc nhẹ dương, nghĩa là khi lãi suất FED tăng, giá ETH có xu hướng tăng nhẹ.

Tuy nhiên, độ dốc rất nhỏ và hệ số tương quan chỉ 0.11 → mối quan hệ gần như không đáng kể.

Lãi suất FED là yếu tố vĩ mô tác động mạnh tới thị trường tài chính truyền thống, nhưng với crypto (đặc biệt là ETH), tác động gián tiếp và chậm trễ

trong thực tế : khi fed tăng lãi suất các nhà đầu tư có xu hướng rút vốn khỏi các tài sản rủi ro cao (như chứng khoán công nghệ, tiền điện tử) để chuyển sang các tài sản an toàn hơn (như trái phiếu chính phủ). Điều này có thể tạo ra áp lực giảm giá cho ETH.

 Lãi suất tăng làm tăng chi phí vay vốn. Điều này có thể ảnh hưởng đến khả năng tài trợ của các công ty công nghệ và các dự án trong hệ sinh thái Ethereum, từ đó làm giảm sự phát triển và giá trị của chúng, nhưng mối quan hệ này không phải lúc nào củng tuyến tính 

In [68]:
import plotly.express as px# --- 1. Resample theo tháng ---
df_month = df.resample('M', on='timestamp_utc').agg({
    'close_eth': 'last',
    'dxy': 'last'
}).reset_index()
# tính hệ số tương quan 
corr_fed = df_month['close_eth'].corr(df_month['dxy'])
print(f"Hệ số tương quan ETH & chỉ số DXY: {corr_fed:.2f}")


# --- 4. Scatter plot ETH vs Fed Funds ---
fig2 = px.scatter(
    df_month, x='dxy', y='close_eth',
    trendline='ols',
    title=f'Mối quan hệ ETH và chỉ số dxy (Corr={corr_fed:.2f})',
    labels={'dxy': 'Chỉ số dxy', 'close_eth': 'Giá ETH (USD)'},
    #template='plotly_dark'
)
fig2.show()


Hệ số tương quan ETH & chỉ số DXY: -0.15


C:\Users\HI\AppData\Local\Temp\ipykernel_20752\1470672476.py:2: FutureWarning:

'M' is deprecated and will be removed in a future version, please use 'ME' instead.



dxy là chỉ số đo sức mạnh đồng usd so với 6 đồng tiền chính như euro,yên, bound,đô la canada,franc thụy sĩ hay korona thụy điển 

khi mà cục dữ trữ liên bang fed tăng lãi suất , usd trở nên hấp dẫn hơn vì lợi suất cao --> dxy tăng và theo số liệu kinh tế mỹ gdp tăng, việc làm tăng usd trở nên mạnh hơn , khi usd mạnh lên thì dòng tiền rút khỏi thị trường crypto và đầu tư vào usd nhiều hơn nên giá eth bị giảm gián tiếp, còn thực chât eth được giao dịch bằng usdt , 1usdt xấp xỉ 1 usd và usdt là stablecoin ko bị ảnh hưởng như trong thị trường ngoại hối